In [1]:
def Read_data(data):
    
    dataX = scipy.io.loadmat('X_' + data + '.mat')
    dataY = scipy.io.loadmat('Y_' + data + '.mat')
    X = dataX['temp']
    Y = dataY['temp']
    
    return X, Y

In [2]:
def Create_D(X,Y,p,c):
    
    n_X = len(X[0,0])
    n_Y = len(Y[0,0])
    T = len(X[0])
    D = np.zeros((T,n_X+1,n_Y+1))
    cp = c ** p
    cp2 = cp/2
    
    for t in range(T):
        for i in range(n_X+1):
            X_exists = True
            if i == n_X:
                X_exists = False
            elif any(np.isnan(X[:,t,i])):
                X_exists = False
            
            for j in range(n_Y+1):
                Y_exists = True
                if j == n_Y:
                    Y_exists = False
                elif any(np.isnan(Y[:,t,j])):
                    Y_exists = False
                
                if X_exists and Y_exists:
                    D[t,i,j] = min(cp, (np.linalg.norm(X[:,t,i] - Y[:,t,j], ord=p)) ** p)
                    
                elif not X_exists and not Y_exists:
                    pass
                    
                else:
                    D[t,i,j] = cp2
    
    return D

In [3]:
def Update_W(D, beta):
    
    n_X = len(D) - 1
    n_Y = len(D[0]) - 1
    
    temp1 = D[0:n_X,0:n_Y] + beta
    temp2 = np.transpose(np.tile(D[0:n_X,n_Y], (n_X,1)))
    temp3 = np.tile(D[n_X,0:n_Y], (n_Y,1))
    temp4 = np.zeros((n_Y,n_X))

    cost = np.block([[temp1, temp2],[temp3, temp4]])
    
    row_ind, col_ind = linear_sum_assignment(cost)
    
    row_ind = [n_X if x>n_X else x for x in row_ind]
    col_ind = [n_Y if x>n_Y else x for x in col_ind]
    
    W = np.zeros((n_X + 1, n_Y + 1))
    W[row_ind,col_ind] = 1
    W[n_X,n_Y] = 0
    
    return W

In [4]:
def Get_upper_bound(D, switching_penalty):
    
    ############ Get input and def constants ###################################################
    n_X = len(D[0]) - 1
    n_Y = len(D[0,0]) - 1
    T = len(D)
    
    ############ Initilization #################################################################
    W = np.zeros((T, n_X+1, n_Y+1))

    for iterations in range(T):
        
        ############ update W ##################################################################
        W_prev = np.copy(W)
        beta = switching_penalty * (np.ones((n_X, n_Y)) - W_prev[1,0:n_X,0:n_Y] -10**-3) 
        W[0,:,:] = Update_W(D[0,:,:], beta)
        
        for t in range(1,T-1):
            beta = switching_penalty * (np.ones((n_X, n_Y)) - W_prev[t-1,0:n_X,0:n_Y] - W_prev[t+1,0:n_X,0:n_Y] -10**-3)
            W[t,:,:] = Update_W(D[t,:,:], beta)
           
        beta = switching_penalty * (np.ones((n_X, n_Y)) - W_prev[T-2,0:n_X,0:n_Y] -10**-3)
        W[T-1,:,:] = Update_W(D[T-1,:,:], beta)
        
        ############ Calculate subgradients and objective value ################################        
        if (W == W_prev).all():
            break
            
    switches = 0
    for t in range(0,T-1):
        switches += np.absolute(W[t,0:n_X,0:n_Y] - W[t+1,0:n_X,0:n_Y]).sum()

    objective = (D * W).sum() + switching_penalty * switches
        
    return objective

In [5]:
def Subgrad_Opt(X, Y, p, c, mu, max_iter):
    
    ############ Get input and def constants ###################################################
    D = Create_D(X,Y,p,c)
    switching_penalty = (mu ** p)/2
    n_X = len(X[0,0])
    n_Y = len(Y[0,0])
    T = len(X[0])

    ############ Initilization #################################################################
    W = np.zeros((T, n_X+1, n_Y+1))
    dual_var = 0.5*switching_penalty*np.ones((T-1, n_X, n_Y))
    delta_W = np.zeros((T-1, n_X, n_Y))
    MDS = 0.5*np.ones((T-1, n_X, n_Y))
    
    ############### CALCULATE UPPER BOUND ######################################################
    upper_bound = Get_upper_bound(D, switching_penalty)
    lower_bound = -np.inf
    
    last_improvement = 0
    alpha = 0.7
    lagrangean_obj = lower_bound
    
    for iterations in range(max_iter):
        
        delta_lambda = 2*dual_var - switching_penalty
        ############ update W ##################################################################
        beta = delta_lambda[0,:,:]
        W[0,:,:] = Update_W(D[0,:,:], beta)
        
        for t in range(1,T-1):
            beta = -delta_lambda[t-1,:,:] + delta_lambda[t,:,:]
            W[t,:,:] = Update_W(D[t,:,:], beta)
           
        beta = - delta_lambda[T-2,:,:]
        W[T-1,:,:] = Update_W(D[T-1,:,:], beta)
        
        for t in range(1,T):
            delta_W[t-1,:,:] = W[t-1,0:n_X,0:n_Y] - W[t,0:n_X,0:n_Y]
        
        ############ Calculate subgradients and objective value ################################
        subgradient = np.copy(delta_W)
        lagrangean_obj_prev = lagrangean_obj
        lagrangean_obj = (D * W).sum() + (delta_lambda*subgradient).sum()
         
        ############ Check special cases for projection ########################################   
        if (dual_var == 0).any() or (dual_var == switching_penalty).any():
            ind1 = np.argwhere(dual_var == 0)
            ind2 = np.argwhere(dual_var == switching_penalty)

            for i in ind1:
                if subgradient[i[0], i[1], i[2]] < 0:
                    subgradient[i[0], i[1], i[2]] = 0
                if MDS[i[0], i[1], i[2]] < 0:
                    MDS[i[0], i[1], i[2]] = 0
                    
            for i in ind2:
                if subgradient[i[0], i[1], i[2]] > 0:
                    subgradient[i[0], i[1], i[2]] = 0  
                if MDS[i[0], i[1], i[2]] > 0:
                    MDS[i[0], i[1], i[2]] = 0  
        
        if (subgradient==0).all():
            lower_bound = lagrangean_obj
            break
        
        if (MDS==0).all():
            if lagrangean_obj > lower_bound:
                lower_bound = lagrangean_obj 
            break
            
        ############ Modified Deflected Subgradient #############################################
        theta = max(0, -(subgradient*MDS).sum() / (np.linalg.norm(subgradient)*np.linalg.norm(MDS)))
        sigma = 1/(2-theta)
        
        gamma_ADS = np.linalg.norm(subgradient)/np.linalg.norm(MDS)
        gamma_MGT = max(0, -sigma * (subgradient*MDS).sum() / (MDS**2).sum())
        gamma_MDS = (1-theta) * gamma_MGT + theta * gamma_ADS
        MDS = subgradient + gamma_MDS * MDS

        step_size = alpha*(upper_bound - lagrangean_obj)/((MDS**2).sum())
        
        dual_var = dual_var + step_size * MDS
        dual_var[dual_var<0] = 0
        dual_var[dual_var>switching_penalty] = switching_penalty
            
        ############### Terminate and update Alpha ####################### 
        if lagrangean_obj > lower_bound:
            prev_lb = lower_bound
            lower_bound = lagrangean_obj 
            if lower_bound - prev_lb < prev_lb*(10**-6):
                small += 1
            else:
                small = 0
                
            last_improvement = 0
            
        if ((upper_bound - lower_bound) < 0.5) or (alpha < 10**-3) or (small == 3): 
            break
            
        if last_improvement >= 20:
            alpha *= 0.5
            last_improvement = 0

        last_improvement += 1
    
    return lower_bound, upper_bound

In [6]:
def Initilization(D, switching_penalty):
    
    ############ Get input and def constants ###################################################
    n_X = len(D[0]) - 1
    n_Y = len(D[0,0]) - 1
    T = len(D)
    
    ############ Initilization #################################################################
    W = np.zeros((T, n_X+1, n_Y+1))
    H = np.zeros((T-1, n_X, n_Y))
    S_1 = np.zeros((T-1, n_X, n_Y))
    S_2 = np.zeros((T-1, n_X, n_Y))
    
    for iterations in range(T):
    
        ############ update W ##################################################################
        W_prev = np.copy(W)

        beta = switching_penalty * (np.ones((n_X, n_Y)) - W_prev[1,0:n_X,0:n_Y] - 10**-3)
        W[0,:,:] = Update_W(D[0,:,:], beta)
    
        for t in range(1,T-1):
            beta = switching_penalty * (np.ones((n_X, n_Y)) - W_prev[t-1,0:n_X,0:n_Y] - W_prev[t+1,0:n_X,0:n_Y] - 10**-3)
            W[t,:,:] = Update_W(D[t,:,:], beta)

        beta = switching_penalty * (np.ones((n_X, n_Y)) - W_prev[T-2,0:n_X,0:n_Y] - 10**-3)
        W[T-1,:,:] = Update_W(D[T-1,:,:], beta)
        
        ############ Calculate subgradients and objective value ##################### 
        if (W == W_prev).all():
            break
        
    for t in range(T-1):
        H[t,:,:] = np.absolute( W[t,0:n_X,0:n_Y] - W[t+1,0:n_X,0:n_Y] )
        S_1[t,:,:] = H[t,:,:] - W[t,0:n_X,0:n_Y] + W[t+1,0:n_X,0:n_Y]
        S_2[t,:,:] = H[t,:,:] - W[t+1,0:n_X,0:n_Y] + W[t,0:n_X,0:n_Y]
    
    return W, H, S_1, S_2

In [7]:
def ADMM_ordinary(X, Y, p, c, mu, rho, max_iter):

    ### Get input and def constants #######################
    D = Create_D(X,Y,p,c)
    switching_penalty = (mu ** p)/2
    n_X = len(X[0,0])
    n_Y = len(Y[0,0])
    T = len(X[0])
    
    tol = 10**-4
    primal_tol = tol*math.sqrt(n_X*n_Y*(T-1)*2) 
    dual_tol = tol*math.sqrt(n_X*n_Y*(T-1)*3) 

    ### Parameters for varing rho ##########################
    rho_incr = 2
    rho_decr = 2
    rho_tol = 10
    
    ### Initilization ######################################
    W, H, S_1, S_2 = Initilization(D, switching_penalty)
    
    lambda_1 = np.zeros((T-1, n_X, n_Y))
    lambda_2 = np.zeros((T-1, n_X, n_Y))
    
    r = np.zeros((T-1, 2, n_X, n_Y))
    
    for iterations in range(max_iter):
        
        ### update W #######################################
        W_prev = np.copy(W)
        delta_lambda = lambda_1[0,:,:] - lambda_2[0,:,:]
        delta_S = S_1[0,:,:] - S_2[0,:,:]
        rho_part = delta_S - 2*W[1,0:n_X,0:n_Y] + np.ones((n_X,n_Y))
        beta = delta_lambda + rho*rho_part
        W[0,:,:] = Update_W(D[0,:,:], beta)
        
        for t in range(1,T-1):
            W_part = - W[t-1,0:n_X,0:n_Y] - W[t+1,0:n_X,0:n_Y] + np.ones((n_X,n_Y))
            rho_part = - delta_S + 2*W_part
            beta = - delta_lambda + rho*rho_part
            delta_lambda = lambda_1[t,:,:] - lambda_2[t,:,:]
            delta_S = S_1[t,:,:] - S_2[t,:,:]
            beta = beta + delta_lambda + rho*delta_S
            W[t,:,:] = Update_W(D[t,:,:], beta)
            
        rho_part = - delta_S - 2*W[T-2,0:n_X,0:n_Y] + np.ones((n_X,n_Y))
        beta = - delta_lambda + rho*rho_part
        W[T-1,:,:] = Update_W(D[T-1,:,:], beta)
            
        ### update H #######################################
        H_prev = np.copy(H)
        H = switching_penalty * np.ones((T-1,n_X,n_Y))
        H = H - lambda_1 - lambda_2 - rho * S_1 - rho * S_2
        H = - H/(2*rho)
        H[H<0] = 0
        
        ### update S_1, S_2, lambda_1 and lambda_2 #########
        S_1_prev = np.copy(S_1)
        S_2_prev = np.copy(S_2)
        for t in range(T-1):
            delta_W = W[t,0:n_X,0:n_Y] - W[t+1,0:n_X,0:n_Y]
            delta_W_H_1 = delta_W - H[t,:,:]
            delta_W_H_2 = - delta_W - H[t,:,:]
            
            ### update S_1 #################################
            min_S_1 = - lambda_1[t,:,:]/rho - delta_W_H_1
            min_S_1[min_S_1<0] = 0
            S_1[t,:,:] = np.copy(min_S_1)

            ### update S_2 #################################
            min_S_2 = - lambda_2[t,:,:]/rho - delta_W_H_2
            min_S_2[min_S_2<0] = 0
            S_2[t,:,:] = np.copy(min_S_2)
            
            ### update lambda_1 ############################
            lambda_1[t,:,:] = lambda_1[t,:,:] + rho*(delta_W_H_1 + S_1[t,:,:])

            ### update lambda_2 ############################
            lambda_2[t,:,:] = lambda_2[t,:,:] + rho*(delta_W_H_2 + S_2[t,:,:])
            
            r[t,0,:,:] = delta_W_H_1 + S_1[t,:,:]
            r[t,1,:,:] = delta_W_H_2 + S_2[t,:,:]
            
        primal_residual = np.linalg.norm(r) 
        z = np.stack((2*W[1:T,0:n_X,0:n_Y], S_1, S_2))
        z_prev = np.stack((2*W_prev[1:T,0:n_X,0:n_Y], S_1_prev, S_2_prev))
        dual_residual = rho*np.linalg.norm((z-z_prev))

        if primal_residual < primal_tol and dual_residual < dual_tol:
            if False:
                print('Number of iteraions: ', iterations+1)
                print('Primal residual: ', primal_residual, 'Primal tolerance: ' , primal_tol)
                print('Dual residual: ', dual_residual, 'Dual tolerance: ' , dual_tol)
                print('W is binary: ', ((W==0) | (W==1)).all())
                # print('r is zero: ', ((r==0)).all())
            break
            
        if primal_residual > rho_tol*dual_residual:
            rho = rho_incr*rho
        elif dual_residual > rho_tol*primal_residual:
            rho = rho/rho_decr
        
    ### Calculate final value #######################################
    objective_value = (D * W).sum() + switching_penalty * H.sum()
    
    return objective_value

In [ ]:
R